****Hito 2****


In [1]:
# importar librerías

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split


In [2]:
# importar datos de la carpeta data
DATA_DIR = Path('./data').resolve()

df = pd.read_excel('./data/base_indices_2005-2023.xlsx', sheet_name='BaseINDICES_2005_2023')

In [3]:
df.head()

,Año,Cód. Institución,Nombre Institución,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,...,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula primer año extranjeros,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula total extranjeros,Matrícula Total,Códgo SIES,Pregrado/Posgrado
0,2023,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,68.0,82.0,2.0,150,302.0,392.0,5.0,694,I70S1C1J1V2,Pregrado
1,2023,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,54.0,45.0,10.0,99,191.0,205.0,23.0,396,I70S1C814J1V1,Pregrado
2,2023,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,353.0,234.0,16.0,587,1535.0,1126.0,52.0,2661,I70S1C58J1V1,Pregrado
3,2023,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,101.0,86.0,10.0,187,395.0,358.0,27.0,753,I70S1C816J1V1,Pregrado
4,2023,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,76.0,56.0,4.0,132,378.0,268.0,6.0,646,I70S1C47J1V1,Pregrado


In [4]:
df.shape

(218773, 60)

Según el enfoque que llegamos del hito 1
donde queremos responder
* ¿Se puede predecir el puntaje de corte esperado para el próximo año en función de las tendencias históricas?
* ¿Existen grupos con características comunes?
* ¿Se pueden agrupar instituciones que tienen perfiles similares de aranceles y puntajes de ingreso?

* Ver el efecto en la matrícula en educación superior (universidades) del ingreso de la gratuidad al sistema de becas
* Ver el efecto del ingreso de la beca vocación de profesor en la matrícula de las carreras de pedagogía
* Ver el efecto de los programas de motivación a que las mujeres se matriculen en carreras STEM, ver una evolución a lo largo de los años

Podemos analizar las columnas según su información y hacer un primer filtro según si aportan a este enfoque


In [5]:
# Columnas del dataframe
print("El DataFrame esta compuesto por las siguientes columnas:")
columns = list(df.columns)
for i in columns:
    print(i)

El DataFrame esta compuesto por las siguientes columnas:
Año
Cód. Institución
Nombre Institución
Tipo Institución
Clasificación1
Clasificación2
Clasificación3
Clasificación4
Clasificación5
Clasificación6
Nombre de la Sede
Comuna donde se imparte la carrera o programa
Nombre Region
Orden Geográfico de la Región (Norte aSur)
Cód. Carrera
Carrera Genérica
Nombre Programa
Mención o Especialidad
Horario
Tipo Programa
Area Conocimiento
idgenerocarrera
Tipo Carrera
IngresoDirecto
Año Inicio Actividades
Nombre del Campus
Duración (en semestres)
Cód. Campus
Cód. Sede
Título
Grado Académico
Máximo Puntaje (promedio matemáticas y lenguaje)
Promedio Puntaje (promedio matemáticas y lenguaje)
Mínimo Puntaje (promedio matemáticas y lenguaje)
Puntaje de corte (primer seleccionado)
Puntaje de corte (promedio de la carrera)
Puntaje de corte (último seleccionado)
Máximo Puntaje NEM
Promedio Puntaje NEM
Mínimo Puntaje NEM
Máximo Puntaje Ranking
Promedio Puntaje Ranking
Mínimo Puntaje Ranking
Nº Alumnos In

Notemos que la base de datos contiene la información de carreras por año, en donde hay data del año sobre las matriculas, la carrera, la institución. Hay columnas que de por sí no nos entregan información relevante a lo que buscamos o se puede extraer de otras columnas, como el orden geografico de la región, las menciones o especialidades, el idgenerocarrera, el nombre del campus, cód. campus y sede, Código SIES.

Por otro lado, si bien nos interesan los puntajes, con la información del premedio basta para nuestro analisis

In [6]:
# Eliminamos esas columnas
df = df.drop(['Nombre de la Sede', 'Orden Geográfico de la Región (Norte aSur)', 'Mención o Especialidad', 'idgenerocarrera', 'Cód. Campus', 'Cód. Sede', 'Códgo SIES'], axis=1)

In [7]:
# Eliminamos máximos
df = df.drop(['Máximo Puntaje (promedio matemáticas y lenguaje)', 'Máximo Puntaje NEM', 'Máximo Puntaje Ranking'], axis=1)

# Eliminamos mínimos
df = df.drop(['Mínimo Puntaje (promedio matemáticas y lenguaje)', 'Mínimo Puntaje NEM', 'Mínimo Puntaje Ranking'], axis=1)

In [8]:
# Identifiquemos los valores únicos por columna
unicos = df.nunique()
print(f"Cantidad de valores únicos por columna: \n \n{unicos} \n ")

Cantidad de valores únicos por columna: 
 
Año                                                      19
Cód. Institución                                        247
Nombre Institución                                      247
Tipo Institución                                          4
Clasificación1                                            6
Clasificación2                                            8
Clasificación3                                            2
Clasificación4                                            6
Clasificación5                                            2
Clasificación6                                            4
Comuna donde se imparte la carrera o programa           127
Nombre Region                                            16
Cód. Carrera                                          33071
Carrera Genérica                                        294
Nombre Programa                                        7935
Horario                                                  

De esto podemos notar que hay columnas que tienen pocos valores unicos, por lo que deben ser una especie de clasificación y por tanto es importanter ver si estas entregan una información relevante.

In [9]:
interes1 = ['Clasificación1', 'Clasificación2', 'Clasificación3', 'Clasificación4', 'Clasificación5', 'Clasificación6']
for column in interes1:
    print(f"Valores únicos de la columna {column}: \n \n{df[column].unique()} \n ")

interes2 = ['Horario', 'Tipo Programa', 'IngresoDirecto', 'Tipo Moneda', "Pregrado/Posgrado"]
for column in interes2:
    print(f"Valores únicos de la columna {column}: \n \n{df[column].unique() } \n ")

Valores únicos de la columna Clasificación1: 
 
['(a) Universidades CRUCH' '(b) Universidades Privadas'
 '(c) Institutos Profesionales' '(d) Centros de Formación Técnica'
 '(e) Centros de Formación Técnica Estatales' '(f) F.F.A.A.'] 
 
Valores únicos de la columna Clasificación2: 
 
['(a) Universidades Estatales CRUCH' '(b) Universidades Privadas CRUCH'
 '(d) Universidades Privadas' '(c) Univ. Privadas Adscritas SUA'
 '(e) Institutos Profesionales' '(f) Centros de Formación Técnica'
 '(g) Centros de Formación Técnica Estatales' '(h) F.F.A.A.'] 
 
Valores únicos de la columna Clasificación3: 
 
['(a) Acreditada' '(b) No Acreditada'] 
 
Valores únicos de la columna Clasificación4: 
 
['(a) Autónoma' '(e) Cerrada' '(b) Licenciamiento' '(e) F.F.A.A.'
 '(c) Examinación' '(d) Supervisión'] 
 
Valores únicos de la columna Clasificación5: 
 
['(a) Adscritas a Gratuidad' '(b) No Adscritas/No Aplica'] 
 
Valores únicos de la columna Clasificación6: 
 
['(a) Subsistema Universitario' '(c) No adsc

Según lo que estamos buscando, dentro de las clasificaciones, la columna "Clasificación2" entrega una sub caracterizacipon de la columna "Clasificación1" por lo que no entrega información relevante según nuestro enfoque. Por otro lado, se puede observar que en la columna "Tipo Moneda" en verdad son loslo 2 opciones que una de ellas está escrita de forma distinta por lo que hay que corregir esa columna. Por último, la comuna de "Pregrado/posgrado" hay que revisar la frecuencia de estas, para elegir si enfocarnos en las matrículas especificas de alguna de ellas o en general.

In [10]:
#df = df.drop(['Clasificación2'], axis=1)

# Reemplazamos los valores de la columna Tipo Moneda a Pesos si la variable es PESOS
df['Tipo Moneda'] = df['Tipo Moneda'].replace('PESOS     ', 'Pesos')
df['Tipo Moneda'] = df['Tipo Moneda'].replace('Pesos     ', 'Pesos')
df['Tipo Moneda'] = df['Tipo Moneda'].replace('pesos     ', 'Pesos')
df['Tipo Moneda'] = df['Tipo Moneda'].replace('UF        ', 'UF')


print(f"Existen {100*sum(df['Pregrado/Posgrado']=='Pregrado')/len(df['Pregrado/Posgrado'])}% datos de pregrado y {100*sum(df['Pregrado/Posgrado']=='Posgrado')/len(df['Pregrado/Posgrado'])}% datos de posgrado")

Existen 83.78684755431429% datos de pregrado y 16.21315244568571% datos de posgrado


Como exite un claro desbalanceo en esta clasificación, solo nos quedamos con las matrículas de pregrado

In [11]:
df = df.drop(df[df["Pregrado/Posgrado"] == "Posgrado"].index)

In [12]:
# Identifiquemos los valores nulos
vacios = df.isnull().sum()
print(f"Porcentaje valores vacios por columna: \n \n{100*vacios/df.shape[0]} \n ")


Porcentaje valores vacios por columna: 
 
Año                                                    0.000000
Cód. Institución                                       0.000000
Nombre Institución                                     0.000000
Tipo Institución                                       0.000000
Clasificación1                                         0.000000
Clasificación2                                         0.000000
Clasificación3                                         0.000000
Clasificación4                                         0.000000
Clasificación5                                         0.000000
Clasificación6                                         0.000000
Comuna donde se imparte la carrera o programa          0.000000
Nombre Region                                          0.000000
Cód. Carrera                                           0.000000
Carrera Genérica                                       0.000000
Nombre Programa                                        0.00000

In [13]:
df.shape

(183303, 47)

Debido a que el porcentaje de nulos en la parte de puntajes es extremadamente alta, este dataset no nos permite realizar un estudio por medio de los puntajes, pero si por medio de los matriculados, tipo de institución. 

Se propone complementar este dataset con otro que incluye información de la infraestructutra de las universidades.

In [14]:
df_inmueble = pd.read_excel('./data/indices_institucional_2005_2022.xlsx', sheet_name='Inmuebles')
df_labs = pd.read_excel('./data/indices_institucional_2005_2022.xlsx', sheet_name='Laboratorios y Talleres')

In [15]:
# Columnas del dataframe
print("El DataFrame esta compuesto por las siguientes columnas:")
columns_in = list(df_inmueble.columns)
for i in columns_in:
    print(i)

El DataFrame esta compuesto por las siguientes columnas:
Tipo Institución
Cód. Institución
Nombre Institución
idInstitucion
idSede
Sede
Año Información
N° Inmuebles
M2 Terreno
M2 Construido
M2 Salas
Nº Oficinas
Nº Salas
Conectividad Megas


In [16]:
# Identifiquemos los valores nulos
vacios_in = df_inmueble.isnull().sum()
print(f"Porcentaje valores vacios por columna: \n \n{100*vacios_in/df_inmueble.shape[0]} \n ")

Porcentaje valores vacios por columna: 
 
Tipo Institución       0.000000
Cód. Institución       0.000000
Nombre Institución     0.000000
idInstitucion          0.000000
idSede                 0.000000
Sede                   0.000000
Año Información        0.000000
N° Inmuebles           1.631461
M2 Terreno             2.095140
M2 Construido          1.597115
M2 Salas               4.619612
Nº Oficinas           11.523270
Nº Salas               2.953804
Conectividad Megas    76.867594
dtype: float64 
 


In [17]:
# Eliminamos columnas que no nos interesan
df_inmueble = df_inmueble.drop(['Tipo Institución', 'Nombre Institución', 'idInstitucion', 'idSede', 'Sede', 'Conectividad Megas'], axis=1)

In [18]:
# Columnas del dataframe
print("El DataFrame esta compuesto por las siguientes columnas:")
columns_lab = list(df_labs.columns)
for i in columns_lab:
    print(i)

El DataFrame esta compuesto por las siguientes columnas:
Tipo Institución
Cód. Institución
Nombre Institución
idSede
Sede
Año Proceso
Nº Laboratorios
M2 Construido
Nº de PC para alumnos
N° Computadores con Internet


In [19]:
# Identifiquemos los valores nulos
vacios_lab = df_labs.isnull().sum()
print(f"Porcentaje valores vacios por columna: \n \n{100*vacios_lab/df_labs.shape[0]} \n ")

Porcentaje valores vacios por columna: 
 
Tipo Institución                0.000000
Cód. Institución                0.000000
Nombre Institución              0.000000
idSede                          0.000000
Sede                            0.000000
Año Proceso                     0.000000
Nº Laboratorios                 0.987965
M2 Construido                   1.508892
Nº de PC para alumnos           3.646488
N° Computadores con Internet    3.969822
dtype: float64 
 


In [20]:
# Eliminamos columnas que no nos interesan
df_labs = df_labs.drop(['Tipo Institución', 'Nombre Institución', 'idSede', 'Sede'], axis=1)

In [21]:
# Arreglamos columna Año
df_inmueble = df_inmueble.rename(columns={'Año Información':'Año'})
df_labs = df_labs.rename(columns={'Año Proceso':'Año'})

df_inmueble['Año'] = pd.to_datetime(df_inmueble['Año'], format='%Y')
df_labs['Año'] = pd.to_datetime(df_labs['Año'], format='%Y')
df['Año'] = pd.to_datetime(df['Año'], format='%Y')

In [28]:
# unimos

df_final = pd.merge(df, df_inmueble, on=['Año','Cód. Institución'], how='left')
df_final = pd.merge(df_final, df_labs, on=['Año', 'Cód. Institución'], how='left')

In [29]:
# Identifiquemos los valores nulos
vacios_fin = df_final.isnull().sum()
print(f"Porcentaje valores vacios por columna: \n \n{100*vacios_fin/df_final.shape[0]} \n ")

Porcentaje valores vacios por columna: 
 
Año                                                    0.000000
Cód. Institución                                       0.000000
Nombre Institución                                     0.000000
Tipo Institución                                       0.000000
Clasificación1                                         0.000000
Clasificación2                                         0.000000
Clasificación3                                         0.000000
Clasificación4                                         0.000000
Clasificación5                                         0.000000
Clasificación6                                         0.000000
Comuna donde se imparte la carrera o programa          0.000000
Nombre Region                                          0.000000
Cód. Carrera                                           0.000000
Carrera Genérica                                       0.000000
Nombre Programa                                        0.00000